# Detecting json schemas

In [49]:
import json
import os
import pandas as pd
import zipfile

In [51]:
def detect_schema_from_json(file):
    """Detects the schema of a JSON file loaded from an open file object."""
    print('detecting schema from ', file)
    data = json.load(file)
    df = pd.json_normalize(data)
    return ', '.join(df.columns)  # Returns a set of column names (schema)

def detect_schema_from_zip(zip_file_path):
    """Detects schemas of JSON files within a .zip archive and returns a combined schema."""
    combined_schema = {}
    print('unzipping ', zip_file_path)
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        for filename in z.namelist():
            print(filename)
            if filename.endswith('.json'):
                with z.open(filename) as file:
                    schema = detect_schema_from_json(file)
                    combined_schema[filename] = schema
    return combined_schema

def aggregate_schemas(directory_path):
    """Iterates over all .zip files in a directory to detect and aggregate schemas."""
    all_schemas = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.zip'):
            zip_file_path = os.path.join(directory_path, filename)
            schema = detect_schema_from_zip(zip_file_path)
            all_schemas.append(schema)
    
    return all_schemas

In [53]:
import os
os.getcwd()

'H:\\002_Studies\\02_Magister_WNE\\3_sem\\2_BigData\\Project\\justjoinit_scripts'

In [59]:
# Example usage
# directory_path = 'justjoinit_sample'
directory_path = 'data'
all_schemas = aggregate_schemas(directory_path)

unzipping  data\archive.zip
2021-10/2021-10-23.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-23.json' mode='r' compress_type=deflate>
2021-10/2021-10-24.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-24.json' mode='r' compress_type=deflate>
2021-10/2021-10-25.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-25.json' mode='r' compress_type=deflate>
2021-10/2021-10-26.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-26.json' mode='r' compress_type=deflate>
2021-10/2021-10-27.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-27.json' mode='r' compress_type=deflate>
2021-10/2021-10-28.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-28.json' mode='r' compress_type=deflate>
2021-10/2021-10-29.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-29.json' mode='r' compress_type=deflate>
2021-10/2021-10-30.json
detecting schema from  <zipfile.ZipExt

In [61]:
aggregate_schemas(directory_path)

unzipping  data\archive.zip
2021-10/2021-10-23.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-23.json' mode='r' compress_type=deflate>
2021-10/2021-10-24.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-24.json' mode='r' compress_type=deflate>
2021-10/2021-10-25.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-25.json' mode='r' compress_type=deflate>
2021-10/2021-10-26.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-26.json' mode='r' compress_type=deflate>
2021-10/2021-10-27.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-27.json' mode='r' compress_type=deflate>
2021-10/2021-10-28.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-28.json' mode='r' compress_type=deflate>
2021-10/2021-10-29.json
detecting schema from  <zipfile.ZipExtFile name='2021-10/2021-10-29.json' mode='r' compress_type=deflate>
2021-10/2021-10-30.json
detecting schema from  <zipfile.ZipExt

[{'2021-10/2021-10-23.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
  '2021-10/2021-10-24.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
  '2021-10/2021-10-25.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
  '2021-10/2021-10-26.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, la

In [62]:
data_dict = all_schemas[0]
data_for_df = [(key.split('/')[1].replace('.json', ''), value) for key, value in data_dict.items()]
df = pd.DataFrame(data_for_df, columns=['Date', 'Schema'])
df['Date'] = pd.to_datetime(df['Date'])

df.head()

,Date,Schema
0,2021-10-23,"title, street, city, country_code, address_tex..."
1,2021-10-24,"title, street, city, country_code, address_tex..."
2,2021-10-25,"title, street, city, country_code, address_tex..."
3,2021-10-26,"title, street, city, country_code, address_tex..."
4,2021-10-27,"title, street, city, country_code, address_tex..."


In [70]:
min_max_dates = df.groupby('Schema').agg(Min_Date=('Date', 'min'), Max_Date=('Date', 'max')).reset_index()

print(min_max_dates)

                                              Schema   Min_Date   Max_Date
0  title, street, city, country_code, address_tex... 2021-10-23 2022-02-27
1  title, street, city, country_code, address_tex... 2022-04-14 2022-04-26
2  title, street, city, country_code, address_tex... 2022-04-27 2023-09-25
3  title, street, city, country_code, address_tex... 2022-02-28 2022-04-13


In [81]:
data_dict['2021-10/2021-10-23.json']

'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote'

In [74]:
data_dict['2022-02/2022-02-28.json']

'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, open_to_hire_ukrainians, id, employment_types, company_logo_url, skills, remote'

In [76]:
data_dict['2022-04/2022-04-14.json']

'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, open_to_hire_ukrainians, id, display_offer, employment_types, company_logo_url, skills, remote, multilocation'

In [ ]:
data_dict['2022-04/2022-04-28.json']

In [83]:
data_dict

{'2021-10/2021-10-23.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
 '2021-10/2021-10-24.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
 '2021-10/2021-10-25.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitude, longitude, published_at, remote_interview, id, employment_types, company_logo_url, skills, remote',
 '2021-10/2021-10-26.json': 'title, street, city, country_code, address_text, marker_icon, workplace_type, company_name, company_url, company_size, experience_level, latitu

# Convert to parquet and add metadata

In [112]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import json
from datetime import datetime

def convert_json_to_parquet_with_date(input_dir, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Iterate over all files in the input directory recursively
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            print('converting', file)
            if file.endswith('.json'):
                # Construct the full file path
                file_path = os.path.join(root, file)
                # Extract the date from the file name (assuming YYYY-MM-DD.json format)
                date_str = os.path.splitext(file)[0]
                date = datetime.strptime(date_str, "%Y-%m-%d")
                
                # Read the JSON file
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                # Convert JSON data to a DataFrame
                df = pd.json_normalize(data)
                # Add a date column from the file name
                df['date'] = date
                
                # Define the output path (preserving the input structure)
                rel_path = os.path.relpath(root, input_dir)
                output_path = os.path.join(output_dir, rel_path, f"{date_str}.parquet")
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
                # Convert DataFrame to Parquet
                table = pa.Table.from_pandas(df)
                pq.write_table(table, output_path)

# Example usage
input_dir = 'pq_data_input'
output_dir = 'pq_data_output'
convert_json_to_parquet_with_date(input_dir, output_dir)


converting archiv.zip
converting 2021-10-23.json
converting 2021-10-24.json
converting 2021-10-25.json
converting 2021-10-26.json
converting 2021-10-27.json
converting 2021-10-28.json
converting 2021-10-29.json
converting 2021-10-30.json
converting 2021-10-31.json
converting 2021-11-01.json
converting 2021-11-02.json
converting 2021-11-03.json
converting 2021-11-04.json
converting 2021-11-05.json
converting 2021-11-06.json
converting 2021-11-07.json
converting 2021-11-08.json
converting 2021-11-09.json
converting 2021-11-10.json
converting 2021-11-11.json
converting 2021-11-12.json
converting 2021-11-13.json
converting 2021-11-14.json
converting 2021-11-15.json
converting 2021-11-16.json
converting 2021-11-17.json
converting 2021-11-18.json
converting 2021-11-19.json
converting 2021-11-20.json
converting 2021-11-21.json
converting 2021-11-22.json
converting 2021-11-23.json
converting 2021-11-24.json
converting 2021-11-25.json
converting 2021-11-26.json
converting 2021-11-27.json
conver

In [108]:
2+2

4

In [93]:
output_dir

'pq_data/'

In [101]:
os.getcwd()

'H:\\002_Studies\\02_Magister_WNE\\3_sem\\2_BigData\\Project\\justjoinit_scripts'